<a href="https://colab.research.google.com/github/dimaszuda/LFP_Battery_SoC_Prediction/blob/main/build_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
dataframe = pd.ExcelFile("ChargeDischarge 18650.xlsx").parse('record')

In [ ]:
dataframe.head(2)

In [ ]:
dataframe.drop(dataframe.columns[[1, 3,  4, 6, 10, 11, 12, 13, 14]], axis=1, inplace=True)

In [ ]:
dataframe = dataframe.rename(columns= {'Step Type' : 'Step_Type'})

In [ ]:
dataframe.isna().sum()

In [ ]:
dataframe = dataframe.dropna()

In [ ]:
dataframe = dataframe.drop_duplicates()

In [ ]:
dataframe.head(2)

In [ ]:
def outlier(data, column):
  Q1 = data[column].quantile(0.25)
  Q3 = data[column].quantile(0.75)
  IQR = Q3 - Q1
  lwr = Q1 - (1.5*IQR)
  upr = Q3 + (1.5*IQR)
  return lwr, upr

In [ ]:
font = {'family': 'serif',
        'color': 'darkred',
        'weight': 'bold'
        }

In [ ]:
def plot_outlier(data, column):
  fig, ax = plt.subplots(figsize=(6, 4))
  ax.set_title('Data Outlier')
  ax.boxplot(data)
  ax.set_aspect(1.5, anchor='C')
  ax.set_xticks([y+1 for y in range(len(data))], labels=['Original Data', 'removed outlier', 'fully removed'])
  ax.set_xlabel(column, fontdict=font)
  plt.show()

In [ ]:
lwr, upr = outlier(dataframe, 'Voltage(V)')
dataframe1 = dataframe[~((dataframe['Voltage(V)'] < lwr) | (dataframe['Voltage(V)'] > upr))]

In [ ]:
lwr, upr = outlier(dataframe1, 'Voltage(V)')
dataframe2 = dataframe1[~((dataframe1['Voltage(V)'] < lwr) | (dataframe1['Voltage(V)'] > upr))]

In [ ]:
lwr, upr = outlier(dataframe2, 'Voltage(V)')
dataframe3 = dataframe2[~((dataframe2['Voltage(V)'] < lwr) | (dataframe2['Voltage(V)'] > upr))]

In [ ]:
df1 = dataframe1['Voltage(V)'].dropna()
df2 = dataframe2['Voltage(V)'].dropna()
df3 = dataframe3['Voltage(V)'].dropna()
sample = [df1, df2, df3]

In [ ]:
plot_outlier(sample, 'Voltage(V)')

In [ ]:
charge = dataframe3[(dataframe3['Step_Type'] == 'CC_Chg')]

In [ ]:
discharge = dataframe3[(dataframe3['Step_Type'] == 'CC_DChg')]

In [ ]:
cc_dc = charge.append(discharge)

In [ ]:
c_0_5 = cc_dc[(cc_dc['C_Rate']<1.000000)]
c_1 = cc_dc[(cc_dc['C_Rate']>1.000000) & (cc_dc['C_Rate']<2.000000)]
c_2 = cc_dc[(cc_dc['C_Rate']>2.000000) & (cc_dc['C_Rate']<3.000000)]
c_3 = cc_dc[(cc_dc['C_Rate']>3.000000)]

In [ ]:
len(c_0_5)

In [ ]:
len(c_1)

In [ ]:
len(c_2)

In [ ]:
len(c_3)

In [ ]:
columns = ['CycleID',
           'Current(mA)',
           'Voltage(V)',
           'Cap(mAh)']

In [ ]:
dataframe_fitur = cc_dc[columns]
dataframe_label = cc_dc['SoC (%)'].values.reshape(-1,1)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
df_f = scaler.fit_transform(dataframe_fitur)
dataframe_fitur = pd.DataFrame(df_f, columns=columns)

In [ ]:
df_l = scaler.fit_transform(dataframe_label)
dataframe_label = pd.DataFrame(df_l, columns=['SoC (%)'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataframe_fitur, dataframe_label, test_size=0.14, random_state=472, shuffle=False)

In [ ]:
def build_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(4)),
      tf.keras.layers.Dense(16, activation='relu'),
      tf.keras.layers.Dense(8, activation='relu'),
      tf.keras.layers.Dense(4, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  optimizer = tf.keras.optimizers.Adamax(0.0001)
  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
data_train = model.fit(X_train, y_train, epochs=1400, batch_size=1000, shuffle=False, validation_split=0.2)

In [ ]:
def plot_history(history, title, xlabel, ylabel):
  plt.figure(figsize=(10, 6))
  plt.plot(history.history['loss'], 'r')
  plt.plot(history.history['mse'], 'b')
  plt.plot(history.history['val_loss'], 'g')
  plt.plot(history.history['val_mse'], 'y')
  plt.title(title)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.legend(['mae', 'mse', 'val_mae', 'val_mse'], loc='upper right')
  plt.show()

In [ ]:
plot_history(data_train, 'Plot Training', 'epochs', 'mae, mse, val_mae dan val_mse')

In [ ]:
data_test = model.evaluate(X_test, y_test)

In [ ]:
data_prediksi = model.predict(dataframe_fitur)

In [ ]:
predicted_data = pd.DataFrame(data_prediksi, columns=['Predicted'])

In [ ]:
Voltage = cc_dc['Voltage(V)']

In [ ]:
data_Voltage = pd.DataFrame(Voltage, columns=['Voltage(V)'])

In [ ]:
predicted_SoC = scaler.inverse_transform(predicted_data)

In [ ]:
def plot_prediksi(step, voltage, judul, sb_x, sb_y):
  plt.figure(figsize=(15, 6))
  plt.scatter(step, voltage, c='#fc0303')
  plt.rcParams['lines.linewidth'] == 43
  plt.rcParams['lines.linestyle'] == '-'
  plt.rcParams['lines.markersize']**0.5
  plt.title(judul)
  plt.xlabel(sb_x)
  plt.ylabel(sb_y)
  plt.show()
  return plt

In [ ]:
plot_prediksi(predicted_SoC, data_Voltage, 'SoC', 'SoC Prediksi', 'Voltage(V)')